<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_DEBUG_200000_(Work_in_Progress_incorporar_PAWS_de_a_poco)_Copy_of_PyTorch_XLA_ResNet18_CIFAR10_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 145.0 MB 11 kB/s 
     |████████████████████████████████| 57 kB 2.9 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.278 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [2]:
!pip uninstall torch -y
!pip install torch==1.8.1

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
     |████████████████████████████████| 804.1 MB 1.9 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.


In [3]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms
import argparse
import yaml
import pprint
import logging
import os

In [42]:
from src.data_manager import GaussianBlur

In [7]:
pip install -U PyYAML

     |████████████████████████████████| 636 kB 5.4 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
pwd

'/content'

In [5]:
!git clone -b feature/DR-images-v2 https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 351, done.
remote: Counting objects: 100% (351/351), done.
remote: Compressing objects: 100% (230/230), done.
remote: Total 351 (delta 211), reused 247 (delta 118), pack-reused 0
Receiving objects: 100% (351/351), 1.11 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (211/211), done.


In [6]:
cd suncet

/content/suncet


In [8]:
!mkdir datasets
!mkdir datasets/dr
!mkdir logs

In [9]:
!python download.py -d sample@2000

Downloading...
From: https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F
To: /content/suncet/datasets/dr/sample@2000.zip
214MB [00:01, 141MB/s]


In [10]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [11]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/dr_train.yaml'
])

In [12]:
fname = args.fname
sel = args.sel

In [13]:
logging.basicConfig()
logger = logging.getLogger()

logger.info(f'called-params {sel} {fname}')

In [14]:
# -- load script params
params = None
with open(fname, 'r') as y_file:
    #params = yaml.load(y_file, Loader=yaml.FullLoader)
    params = yaml.load(y_file)
    logger.info('loaded params...')
    #if rank == 0:
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(params)

{   'criterion': {   'classes_per_batch': 2,
                     'me_max': True,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 8,
                     'supervised_views': 1,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 32},
    'data': {   'color_jitter_strength': 1.0,
                'data_seed': None,
                'dataset': 'dr',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                's_image_folder': 'dr/sample@2000/',
                'subset_path': 'dr_subsets',
                'u_image_folder': 'dr/sample@2000/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.9},
    'logging': {'folder': 'logs/', 'write_tag': 'paws'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': False,
               

In [15]:
args = params

In [16]:
# Define Parameters
FLAGS = {}
FLAGS['data_dir'] = "/tmp/cifar"
FLAGS['batch_size'] = 32
FLAGS['num_workers'] = 2
FLAGS['learning_rate'] = 0.02
FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 10
FLAGS['num_cores'] = 8 if os.environ.get('TPU_NAME', None) else 1
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False

In [17]:
FLAGS['model_name'] = args["meta"]["model_name"]

In [18]:
model_name = args["meta"]["model_name"]

In [19]:
output_dim = args["meta"]["output_dim"]
multicrop = args["data"]["multicrop"]

In [20]:
# -- CRITERTION
reg = args["criterion"]["me_max"]
supervised_views = args["criterion"]["supervised_views"]
classes_per_batch = args["criterion"]["classes_per_batch"]
s_batch_size = args["criterion"]["supervised_imgs_per_class"]
u_batch_size = args["criterion"]["unsupervised_batch_size"]
temperature = args["criterion"]["temperature"]
sharpen = args["criterion"]["sharpen"]

In [21]:
# -- DATA
unlabeled_frac = args["data"]["unlabeled_frac"]
color_jitter = args["data"]["color_jitter_strength"]
normalize = args["data"]["normalize"]
root_path = args["data"]["root_path"]
s_image_folder = args["data"]["s_image_folder"]
u_image_folder = args["data"]["u_image_folder"]
dataset_name = args["data"]["dataset"]
subset_path = args["data"]["subset_path"]
unique_classes = args["data"]["unique_classes_per_rank"]
label_smoothing = args["data"]["label_smoothing"]
data_seed = None

In [22]:
crop_scale = (0.14, 1.0) if multicrop > 0 else (0.08, 1.0)
mc_scale = (0.05, 0.14)
mc_size = 96

In [23]:
copy_data = args["meta"]["copy_data"]
use_pred_head = args["meta"]["use_pred_head"]

In [24]:
use_fp16 = args["meta"]["use_fp16"]

In [25]:
# -- OPTIMIZATION
wd = float(args["optimization"]["weight_decay"])
num_epochs = args["optimization"]["epochs"]
warmup = args["optimization"]["warmup"]
start_lr = args["optimization"]["start_lr"]
lr = args["optimization"]["lr"]
final_lr = args["optimization"]["final_lr"]
mom = args["optimization"]["momentum"]
nesterov = args["optimization"]["nesterov"]

In [ ]:
# ----------------------------------------------------------------------- #
#  PASSED IN PARAMS FROM CONFIG FILE
# ----------------------------------------------------------------------- #
# -- META


load_model = args["meta"]["load_checkpoint"]
r_file = args["meta"]["read_checkpoint"]


#device = torch.device(args["meta"]["device"])
device = xm.xla_device()




# -- LOGGING
folder = args["logging"]["folder"]
tag = args["logging"]["write_tag"]
# ----------------------------------------------------------------------- #

# -- init torch distributed backend
#world_size, rank = init_distributed()


In [26]:
import logging
import sys
from collections import OrderedDict
import traceback
import numpy as np

import torch

import src.resnet as resnet
import src.wide_resnet as wide_resnet
from src.utils import (
    gpu_timer,
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)
from src.losses import init_paws_loss, make_labels_matrix
from src.data_manager import init_data, make_transforms, make_multicrop_transform
from src.sgd import SGD
from src.lars import LARS

import torchvision.models as models
from torch.hub import load_state_dict_from_url

#import apex
from torch.nn.parallel import DistributedDataParallel

In [28]:
from src.paws_train import init_model

In [29]:
from src.data_manager import ImageNet

In [30]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

import os
import subprocess
import time

from logging import getLogger

import numpy as np
from math import ceil

import torch

import torchvision.transforms as transforms
import torchvision

import PIL
from PIL import Image
from PIL import ImageFilter

_GLOBAL_SEED = 0
logger = getLogger()


def init_data(
    dataset_name,
    transform,
    init_transform,
    u_batch_size,
    s_batch_size,
    classes_per_batch,
    world_size,
    rank,
    unique_classes=False,
    multicrop_transform=(0, None),
    supervised_views=1,
    root_path=None,
    s_image_folder=None,
    u_image_folder=None,
    image_folder=None,
    training=True,
    copy_data=False,
    stratify=False,
    drop_last=True,
):
    """
    :param dataset_name: ['imagenet', 'cifar10', 'cifar10_fine_tune', 'imagenet_fine_tune']
    :param transform: torchvision transform to apply to each batch of data
    :param init_transform: transform to apply once to all data at the start
    :param u_batch_size: unsupervised batch-size
    :param s_batch_size: supervised batch-size (images per class)
    :param classes_per_batch: num. classes sampled in each supervised batch per gpu
    :param unique_classes: whether each GPU should load different classes
    :param multicrop_transform: number of smaller multi-crop images to return
    :param supervised_views: number of views to generate of each labeled imgs
    :param world_size: number of workers for distributed training
    :param rank: rank of worker in distributed training
    :param root_path: path to the root directory containing all dataset
    :param image_folder: name of folder in 'root_path' containing data to load
    :param training: whether to load training data
    :param stratify: whether to class stratify 'fine_tune' data loaders
    :param copy_data: whether to copy data locally to node at start of training
    """

    if dataset_name == "dr":
        return _init_dr_data(
            transform=transform,
            init_transform=init_transform,
            u_batch_size=u_batch_size,
            s_batch_size=s_batch_size,
            classes_per_batch=classes_per_batch,
            unique_classes=unique_classes,
            multicrop_transform=multicrop_transform,
            supervised_views=supervised_views,
            world_size=world_size,
            rank=rank,
            root_path=root_path,
            s_image_folder=s_image_folder,
            u_image_folder=u_image_folder,
            training=training,
            copy_data=copy_data,
        )

    elif dataset_name == "dr_fine_tune":
        batch_size = s_batch_size
        return _init_imgnt_ft_data(
            transform=transform,
            init_transform=init_transform,
            batch_size=batch_size,
            stratify=stratify,
            classes_per_batch=classes_per_batch,
            unique_classes=unique_classes,
            world_size=world_size,
            rank=rank,
            root_path=root_path,
            image_folder=image_folder,
            training=training,
            drop_last=drop_last,
            copy_data=copy_data,
        )


def _init_dr_data(
    transform,
    init_transform,
    u_batch_size,
    s_batch_size,
    classes_per_batch,
    world_size,
    rank,
    unique_classes=False,
    multicrop_transform=(0, None),
    supervised_views=1,
    root_path="/datasets/",
    s_image_folder="imagenet_full_size/061417/",
    u_image_folder="imagenet_full_size/061417/",
    training=True,
    copy_data=False,
    tar_folder="imagenet_full_size/",
    tar_file="imagenet_full_size-061417.tar",
):
    s_imagedr = ImageDR(
        root=root_path,
        image_folder=s_image_folder,
        tar_folder=tar_folder,
        tar_file=tar_file,
        transform=transform,
        train=training,
        copy_data=copy_data,
    )

    u_imagedr = ImageDR(
        root=root_path,
        image_folder=u_image_folder,
        tar_folder=tar_folder,
        tar_file=tar_file,
        transform=transform,
        train=training,
        copy_data=copy_data,
    )
    logger.info("ImageDR dataset created")

    logger.info("Making unsupervised ImageDR data loader...")
    unsupervised_set = TransImageDR(
        dataset=u_imagedr,
        supervised=False,
        init_transform=init_transform,
        multicrop_transform=multicrop_transform,
        seed=_GLOBAL_SEED,
    )
    unsupervised_sampler = torch.utils.data.distributed.DistributedSampler(
        dataset=unsupervised_set, num_replicas=world_size, rank=rank
    )
    unsupervised_loader = torch.utils.data.DataLoader(
        unsupervised_set,
        sampler=unsupervised_sampler,
        batch_size=u_batch_size,
        drop_last=True,
        pin_memory=True,
        num_workers=8,
    )
    logger.info("ImageDR unsupervised data loader created")

    supervised_sampler, supervised_loader = None, None
    if classes_per_batch > 0 and s_batch_size > 0:
        logger.info("Making supervised ImageDR data loader...")

        supervised_set = TransImageDR(
            dataset=s_imagedr,
            supervised=True,
            supervised_views=supervised_views,
            init_transform=init_transform,
            seed=_GLOBAL_SEED,
        )

        """
        supervised_sampler = ClassStratifiedSampler(
            data_source=supervised_set,
            world_size=world_size,
            rank=rank,
            batch_size=s_batch_size,
            classes_per_batch=classes_per_batch,
            unique_classes=unique_classes,
            seed=_GLOBAL_SEED,
        )
        supervised_loader = torch.utils.data.DataLoader(
            supervised_set,
            batch_sampler=supervised_sampler,
            pin_memory=True,
            num_workers=8,
        )
        """
        #supervised_set = datasets.ImageFolder(
        #    root=root_path+s_image_folder
        #)

        supervised_sampler = torch.utils.data.distributed.DistributedSampler(
            dataset=supervised_set, num_replicas=world_size, rank=rank
        )

        supervised_loader = torch.utils.data.DataLoader(
            supervised_set,
            sampler=supervised_sampler,
            batch_size=s_batch_size*2, # YOO LE Agrege la Multiplicacion
            drop_last=True,
            pin_memory=True,
            num_workers=8,
        )
        """
        if len(supervised_loader) > 0:
            tmp = ceil(len(unsupervised_loader) / len(supervised_loader))
            supervised_sampler.set_inner_epochs(tmp)
            logger.info(f"supervised-reset-period {tmp}")
        """
        logger.info("ImageDR supervised data loader created")

    return (
        unsupervised_loader,
        unsupervised_sampler,
        supervised_loader,
        supervised_sampler,
    )


def _init_imgnt_ft_data(
    transform,
    init_transform,
    batch_size,
    world_size,
    rank,
    stratify=False,
    classes_per_batch=1,
    unique_classes=False,
    root_path="/datasets/",
    image_folder="imagenet_full_size/061417/",
    training=True,
    copy_data=False,
    drop_last=True,
    tar_folder="imagenet_full_size/",
    tar_file="imagenet_full_size-061417.tar",
):
    imagenet = ImageDR(
        root=root_path,
        image_folder=image_folder,
        tar_folder=tar_folder,
        tar_file=tar_file,
        transform=transform,
        train=training,
        copy_data=copy_data,
    )
    logger.info("ImageNet fine-tune dataset created")
    dataset = TransImageDR(
        dataset=imagenet,
        supervised=True,
        init_transform=init_transform,
        seed=_GLOBAL_SEED,
    )

    if not stratify:
        dist_sampler = torch.utils.data.distributed.DistributedSampler(
            dataset=dataset, num_replicas=world_size, rank=rank
        )
        data_loader = torch.utils.data.DataLoader(
            dataset,
            sampler=dist_sampler,
            batch_size=batch_size,
            drop_last=drop_last,
            pin_memory=True,
            num_workers=8,
        )
    else:
        dist_sampler = ClassStratifiedSampler(
            data_source=dataset,
            world_size=world_size,
            rank=rank,
            batch_size=batch_size,
            classes_per_batch=classes_per_batch,
            seed=_GLOBAL_SEED,
            unique_classes=unique_classes,
        )
        data_loader = torch.utils.data.DataLoader(
            dataset, batch_sampler=dist_sampler, pin_memory=True, num_workers=8
        )

    return (data_loader, dist_sampler)


def make_transforms(
    dataset_name,
    subset_path=None,
    unlabeled_frac=1.0,
    training=True,
    basic_augmentations=False,
    force_center_crop=False,
    crop_scale=(0.08, 1.0),
    color_jitter=1.0,
    normalize=False,
    split_seed=0,
):
    """
    :param dataset_name: ['imagenet', 'cifar10']
    :param subset_path: path to .txt file denoting subset of data to use
    :param unlabeled_frac: fraction of data that is unlabeled
    :param training: whether to load training data
    :param basic_augmentations: whether to use simple data-augmentations
    :param force_center_crop: whether to force use of a center-crop
    :param color_jitter: strength of color-jitter
    :param normalize: whether to normalize color channels
    """

    if "dr" in dataset_name:
        logger.info("making imagenet data transforms")

        # -- file identifying which imagenet labels to keep
        keep_file = None
        if subset_path is not None:
            if unlabeled_frac >= 0:
                keep_file = os.path.join(
                    subset_path, f"{int(unlabeled_frac* 100)}percent.txt"
                )
            else:
                keep_file = os.path.join(subset_path, "val.txt")
            logger.info(f"keep file: {keep_file}")

        return _make_imgnt_transforms(
            unlabel_prob=unlabeled_frac,
            training=training,
            basic=basic_augmentations,
            force_center_crop=force_center_crop,
            normalize=normalize,
            color_distortion=color_jitter,
            scale=crop_scale,
            keep_file=keep_file,
        )


def _make_imgnt_transforms(
    unlabel_prob,
    training=True,
    basic=False,
    force_center_crop=False,
    normalize=False,
    scale=(0.08, 1.0),
    color_distortion=1.0,
    keep_file=None,
):
    """
    Make data transformations

    :param unlabel_prob: probability of sampling unlabeled data point
    :param training: generate data transforms for train (alternativly test)
    :param basic: whether train transforms include more sofisticated transforms
    :param force_center_crop: whether to override settings and apply center crop to image
    :param normalize: whether to normalize image means and stds
    :param scale: random scaling range for image before resizing
    :param color_distortion: strength of color distortion
    :param keep_file: file containing names of images to use for semisupervised
    """

    def get_color_distortion(s=1.0):
        # s is the strength of color distortion.
        color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
        rnd_color_jitter = transforms.RandomApply([color_jitter], p=0.8)
        rnd_gray = transforms.RandomGrayscale(p=0.2)
        color_distort = transforms.Compose([rnd_color_jitter, rnd_gray])
        return color_distort

    logger.debug(
        f"uprob: {unlabel_prob}\t training: {training}\t basic: {basic}\t normalize: {normalize}\t color_distortion: {color_distortion}"
    )
    if training and (not force_center_crop):
        if basic:
            transform = transforms.Compose(
                [
                    transforms.RandomResizedCrop(size=224, scale=scale),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                ]
            )
        else:
            logger.debug("making training (non-basic) transforms")
            transform = transforms.Compose(
                [
                    transforms.RandomResizedCrop(size=224, scale=scale),
                    transforms.RandomHorizontalFlip(),
                    get_color_distortion(s=color_distortion),
                    GaussianBlur(p=0.5),
                    transforms.ToTensor(),
                ]
            )
    else:
        transform = transforms.Compose(
            [
                transforms.Resize(size=256),
                transforms.CenterCrop(size=224),
                transforms.ToTensor(),
            ]
        )

    if normalize:
        transform = transforms.Compose(
            [
                transform,
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ]
        )

    def init_transform(
        root, samples, class_to_idx, seed, keep_file=keep_file, training=training
    ):
        """Transforms applied to dataset at the start of training"""

        new_targets, new_samples = [], []
        if training and (keep_file is not None) and os.path.exists(keep_file):
            logger.info(f"Using {keep_file}")
            with open(keep_file, "r") as rfile:
                for line in rfile:
                    class_name = line.split("_")[0]
                    target = class_to_idx[class_name]
                    img = line.split("\n")[0]
                    new_samples.append((os.path.join(root, class_name, img), target))
                    new_targets.append(target)
        else:
            logger.info("flipping coin to keep labels")
            g = torch.Generator()
            g.manual_seed(seed)
            for sample in samples:
                if torch.bernoulli(torch.tensor(unlabel_prob), generator=g) == 0:
                    target = sample[1]
                    new_samples.append((sample[0], target))
                    new_targets.append(target)

        return np.array(new_targets), np.array(new_samples)

    return transform, init_transform


def make_multicrop_transform(
    dataset_name, num_crops, size, crop_scale, normalize, color_distortion
):
    if "dr" in dataset_name:
        return _make_multicrop_imgnt_transforms(
            num_crops=num_crops,
            size=size,
            scale=crop_scale,
            normalize=normalize,
            color_distortion=color_distortion,
        )


def _make_multicrop_imgnt_transforms(
    num_crops, size=96, scale=(0.05, 0.14), normalize=False, color_distortion=1.0,
):
    def get_color_distortion(s=1.0):
        color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
        rnd_color_jitter = transforms.RandomApply([color_jitter], p=0.8)
        rnd_gray = transforms.RandomGrayscale(p=0.2)
        color_distort = transforms.Compose([rnd_color_jitter, rnd_gray])
        return color_distort

    logger.debug("making multicrop transforms")
    transform = transforms.Compose(
        [
            transforms.RandomResizedCrop(size=size, scale=scale),
            transforms.RandomHorizontalFlip(),
            get_color_distortion(s=color_distortion),
            GaussianBlur(p=0.5),
            transforms.ToTensor(),
        ]
    )

    if normalize:
        transform = transforms.Compose(
            [
                transform,
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ]
        )

    return (num_crops, transform)

In [44]:
class ImageNet(torchvision.datasets.ImageFolder):
    def __init__(
        self,
        root,
        image_folder="imagenet_full_size/061417/",
        tar_folder="imagenet_full_size/",
        tar_file="imagenet_full_size-061417.tar",
        train=True,
        transform=None,
        target_transform=None,
        job_id=None,
        local_rank=None,
        copy_data=True,
    ):
        """
        ImageNet
        Dataset wrapper (can copy data locally to machine)
        :param root: root network directory for ImageNet data
        :param image_folder: path to images inside root network directory
        :param tar_file: zipped image_folder inside root network directory
        :param train: whether to load train data (or validation)
        :param transform: data-augmentations (applied in data-loader)
        :param target_transform: target-transform to apply in data-loader
        :param job_id: scheduler job-id used to create dir on local machine
        :param copy_data: whether to copy data from network file locally
        """

        suffix = "train/" if train else "val/"
        data_path = None
        if copy_data:
            logger.info("copying data locally")
            data_path = copy_imgnt_locally(
                root=root,
                suffix=suffix,
                image_folder=image_folder,
                tar_folder=tar_folder,
                tar_file=tar_file,
                job_id=job_id,
                local_rank=local_rank,
            )
        if (not copy_data) or (data_path is None):
            data_path = os.path.join(root, image_folder, suffix)
        logger.info(f"data-path {data_path}")

        super(ImageNet, self).__init__(
            root=data_path, transform=transform, target_transform=target_transform
        )
        logger.info("Initialized ImageNet")

class ImageDR(torchvision.datasets.ImageFolder):
    def __init__(
        self,
        root,
        image_folder="imagenet_full_size/061417/",
        tar_folder="imagenet_full_size/",
        tar_file="imagenet_full_size-061417.tar",
        train=True,
        transform=None,
        target_transform=None,
        job_id=None,
        local_rank=None,
        copy_data=True,
    ):
        """
        ImageNet

        Dataset wrapper (can copy data locally to machine)

        :param root: root network directory for DR data
        :param image_folder: path to images inside root network directory
        :param tar_file: zipped image_folder inside root network directory
        :param train: whether to load train data (or validation)
        :param transform: data-augmentations (applied in data-loader)
        :param target_transform: target-transform to apply in data-loader
        :param job_id: scheduler job-id used to create dir on local machine
        :param copy_data: whether to copy data from network file locally
        """

        suffix = "train/" if train else "val/"
        data_path = None
        if copy_data:
            logger.info("copying data locally")
            data_path = copy_imgnt_locally(
                root=root,
                suffix=suffix,
                image_folder=image_folder,
                tar_folder=tar_folder,
                tar_file=tar_file,
                job_id=job_id,
                local_rank=local_rank,
            )
        if (not copy_data) or (data_path is None):
            data_path = os.path.join(root, image_folder, suffix)
        logger.info(f"data-path {data_path}")

        super(ImageDR, self).__init__(
            root=data_path, transform=transform, target_transform=target_transform
        )
        logger.info("Initialized ImageDR")


class TransImageDR(ImageNet):
    def __init__(
        self,
        dataset,
        supervised=False,
        supervised_views=1,
        init_transform=None,
        multicrop_transform=(0, None),
        seed=0,
    ):
        """
        TransImageDR

        Dataset that can apply transforms to images on initialization and can
        return multiple transformed copies of the same image in each call
        to __getitem__
        """
        self.dataset = dataset
        self.supervised = supervised
        self.supervised_views = supervised_views
        self.multicrop_transform = multicrop_transform

        print("self.multicrop_transform {}".format(self.multicrop_transform))

        self.targets, self.samples = dataset.targets, dataset.samples
        if self.supervised:
            self.targets, self.samples = init_transform(
                dataset.root, dataset.samples, dataset.class_to_idx, seed
            )
            logger.debug(f"num-labeled {len(self.samples)}")
            mint = None
            self.target_indices = []
            for t in range(len(dataset.classes)):
                indices = np.squeeze(np.argwhere(self.targets == t)).tolist()
                self.target_indices.append(indices)
                mint = len(indices) if mint is None else min(mint, len(indices))
                logger.debug(f"num-labeled target {t} {len(indices)}")
            logger.debug(f"min. labeled indices {mint}")

    @property
    def classes(self):
        return self.dataset.classes

    def __getitem__(self, index):
        target = self.targets[index]
        path = self.samples[index][0]
        img = self.dataset.loader(path)

        if self.dataset.target_transform is not None:
            target = self.dataset.target_transform(target)

        if self.dataset.transform is not None:
            if self.supervised:
                ans = (
                    *[
                        self.dataset.transform(img)
                        for _ in range(self.supervised_views)
                    ],
                    target,
                )
                return ans
            else:
                img_1 = self.dataset.transform(img)
                img_2 = self.dataset.transform(img)

                multicrop, mc_transform = self.multicrop_transform
                if multicrop > 0 and mc_transform is not None:
                    mc_imgs = [mc_transform(img) for _ in range(int(multicrop))]
                    ans = img_1, img_2, *mc_imgs, target
                    return ans

                return img_1, img_2, target

        return img, target


def copy_imgnt_locally(
    root,
    suffix,
    image_folder="imagenet_full_size/061417/",
    tar_folder="imagenet_full_size/",
    tar_file="imagenet_full_size-061417.tar",
    job_id=None,
    local_rank=None,
):
    if job_id is None:
        try:
            job_id = os.environ["SLURM_JOBID"]
        except Exception:
            logger.info("No job-id, will load directly from network file")
            return None

    if local_rank is None:
        try:
            local_rank = int(os.environ["SLURM_LOCALID"])
        except Exception:
            logger.info("No job-id, will load directly from network file")
            return None

    source_file = os.path.join(root, tar_folder, tar_file)
    target = f"/scratch/slurm_tmpdir/{job_id}/"
    target_file = os.path.join(target, tar_file)
    data_path = os.path.join(target, image_folder, suffix)
    logger.info(f"{source_file}\n{target}\n{target_file}\n{data_path}")

    tmp_sgnl_file = os.path.join(target, "copy_signal.txt")

    if not os.path.exists(data_path):
        if local_rank == 0:
            commands = [["tar", "-xf", source_file, "-C", target]]
            for cmnd in commands:
                start_time = time.time()
                logger.info(f"Executing {cmnd}")
                subprocess.run(cmnd)
                logger.info(f"Cmnd took {(time.time()-start_time)/60.} min.")
            with open(tmp_sgnl_file, "+w") as f:
                print("Done copying locally.", file=f)
        else:
            while not os.path.exists(tmp_sgnl_file):
                time.sleep(60)
                logger.info(f"{local_rank}: Checking {tmp_sgnl_file}")

    return data_path


class GaussianBlur(object):
    def __init__(self, p=0.5, radius_min=0.1, radius_max=2.0):
        self.prob = p
        self.radius_min = radius_min
        self.radius_max = radius_max

    def __call__(self, img):
        if torch.bernoulli(torch.tensor(self.prob)) == 0:
            return img

        radius = self.radius_min + torch.rand(1) * (self.radius_max - self.radius_min)
        return img.filter(ImageFilter.GaussianBlur(radius=radius))

In [ ]:
"""
def load_checkpoint(r_path, encoder, opt, scaler, use_fp16=False):
    checkpoint = torch.load(r_path, map_location="cpu")
    epoch = checkpoint["epoch"]

    # -- loading encoder
    encoder.load_state_dict(checkpoint["encoder"])
    logger.info(f"loaded encoder from epoch {epoch}")

    # -- loading optimizer
    opt.load_state_dict(checkpoint["opt"])
    if use_fp16:
        scaler.load_state_dict(checkpoint["amp"])
    logger.info(f"loaded optimizers from epoch {epoch}")
    logger.info(f"read-path: {r_path}")
    del checkpoint
    return encoder, opt, epoch


def init_model(device, model_name="resnet50", use_pred=False, output_dim=128):
    if "wide_resnet" in model_name:
        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)
        hidden_dim = 128
    else:
        encoder = resnet.__dict__[model_name]() 

        # Load pre-trained ResNetImagenNet
        #logger.info("Load pre-trained ResNet ImagenNet weigths ...")
        #state_dict = load_state_dict_from_url('https://download.pytorch.org/models/resnet50-0676ba61.pth',progress=True)
        #log = encoder.load_state_dict(state_dict, strict=False)
        #logger.info(log)
    
        hidden_dim = 2048
        if "w2" in model_name:
            hidden_dim *= 2
        elif "w4" in model_name:
            hidden_dim *= 4

    # -- projection head
    encoder.fc = torch.nn.Sequential(
        OrderedDict(
            [
                ("fc1", torch.nn.Linear(hidden_dim, hidden_dim)),
                ("bn1", torch.nn.BatchNorm1d(hidden_dim)),
                ("relu1", torch.nn.ReLU(inplace=True)),
                ("fc2", torch.nn.Linear(hidden_dim, hidden_dim)),
                ("bn2", torch.nn.BatchNorm1d(hidden_dim)),
                ("relu2", torch.nn.ReLU(inplace=True)),
                ("fc3", torch.nn.Linear(hidden_dim, output_dim)),
            ]
        )
    )

    # -- prediction head
    encoder.pred = None
    if use_pred:
        mx = 4  # 4x bottleneck prediction head
        pred_head = OrderedDict([])
        pred_head["bn1"] = torch.nn.BatchNorm1d(output_dim)
        pred_head["fc1"] = torch.nn.Linear(output_dim, output_dim // mx)
        pred_head["bn2"] = torch.nn.BatchNorm1d(output_dim // mx)
        pred_head["relu"] = torch.nn.ReLU(inplace=True)
        pred_head["fc2"] = torch.nn.Linear(output_dim // mx, output_dim)
        encoder.pred = torch.nn.Sequential(pred_head)

    encoder.to(device)
    logger.info(encoder)
    return encoder


def init_opt(
    encoder,
    iterations_per_epoch,
    start_lr,
    ref_lr,
    ref_mom,
    nesterov,
    warmup,
    num_epochs,
    weight_decay=1e-6,
    final_lr=0.0,
):
    param_groups = [
        {
            "params": (
                p
                for n, p in encoder.named_parameters()
                if ("bias" not in n) and ("bn" not in n)
            )
        },
        {
            "params": (
                p for n, p in encoder.named_parameters() if ("bias" in n) or ("bn" in n)
            ),
            "LARS_exclude": True,
            "weight_decay": 0,
        },
    ]
    optimizer = SGD(
        param_groups,
        weight_decay=weight_decay,
        momentum=0.9,
        nesterov=nesterov,
        lr=ref_lr,
    )
    scheduler = WarmupCosineSchedule(
        optimizer,
        warmup_steps=warmup * iterations_per_epoch,
        start_lr=start_lr,
        ref_lr=ref_lr,
        final_lr=final_lr,
        T_max=num_epochs * iterations_per_epoch,
    )
    optimizer = LARS(optimizer, trust_coefficient=0.001)
    return encoder, optimizer, scheduler
"""

'\ndef load_checkpoint(r_path, encoder, opt, scaler, use_fp16=False):\n    checkpoint = torch.load(r_path, map_location="cpu")\n    epoch = checkpoint["epoch"]\n\n    # -- loading encoder\n    encoder.load_state_dict(checkpoint["encoder"])\n    logger.info(f"loaded encoder from epoch {epoch}")\n\n    # -- loading optimizer\n    opt.load_state_dict(checkpoint["opt"])\n    if use_fp16:\n        scaler.load_state_dict(checkpoint["amp"])\n    logger.info(f"loaded optimizers from epoch {epoch}")\n    logger.info(f"read-path: {r_path}")\n    del checkpoint\n    return encoder, opt, epoch\n\n\ndef init_model(device, model_name="resnet50", use_pred=False, output_dim=128):\n    if "wide_resnet" in model_name:\n        encoder = wide_resnet.__dict__[model_name](dropout_rate=0.0)\n        hidden_dim = 128\n    else:\n        encoder = resnet.__dict__[model_name]() \n\n        # Load pre-trained ResNetImagenNet\n        #logger.info("Load pre-trained ResNet ImagenNet weigths ...")\n        #state_

## PyTorch/XLA ResNet18/CIFAR10 (GPU or TPU)

In [31]:
import argparse
import logging
import os
import time
import gdown
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.utils.utils as xu
import torchvision
from torchvision import datasets, transforms
from src.utils import (
    init_distributed,
    WarmupCosineSchedule,
    CSVLogger,
    AverageMeter,
)
from src.utils import (
    AllGather,
    AllReduce
)

In [35]:
def download(data, url):
    # Download dataset
    import zipfile
    url = url
    output = "{}.zip".format(data)
    gdown.download(url, output, quiet=False)

    # Uncompress dataset
    local_zip = '{}.zip'.format(data)
    zip_ref = zipfile.ZipFile(local_zip, "r")
    zip_ref.extractall()
    zip_ref.close()

In [32]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [33]:
UNLABELED = 'sample@2000'

In [36]:
URL_UNLABELED = data_samples[UNLABELED]
download(UNLABELED, URL_UNLABELED)

Downloading...
From: https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F
To: /content/suncet/sample@2000.zip
214MB [00:03, 71.2MB/s]


Only run the below commented cell if you would like a nightly release

In [ ]:
"""
# PyTorch/XLA GPU Setup (only if GPU runtime)
if os.environ.get('COLAB_GPU', '0') == '1':
  os.environ['GPU_NUM_DEVICES'] = '1'
  os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda/'
"""

"\n# PyTorch/XLA GPU Setup (only if GPU runtime)\nif os.environ.get('COLAB_GPU', '0') == '1':\n  os.environ['GPU_NUM_DEVICES'] = '1'\n  os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda/'\n"

### Define Parameters



In [37]:
UNLABELED

'sample@2000'

In [38]:
multicrop=multicrop
tau=temperature
T=sharpen
me_max=reg


"""
Make semi-supervised PAWS loss

:param multicrop: number of small multi-crop views
:param tau: cosine similarity temperature
:param T: target sharpenning temperature
:param me_max: whether to perform me-max regularization
"""

def sharpen_func(p):
    sharp_p = p**(1./T)
    sharp_p /= torch.sum(sharp_p, dim=1, keepdim=True)
    return sharp_p

def snn(query, supports, labels, tau):
    #device = xm.xla_device() # YOOOOO

    softmax = torch.nn.Softmax(dim=1)
    """ Soft Nearest Neighbours similarity classifier """
    # Step 1: normalize embeddings
    query = torch.nn.functional.normalize(query)

    #query = query.to(device) # YOOOOO

    #print("Query is:::")
    #print(query)

    supports = torch.nn.functional.normalize(supports)

    #labels = labels.to(device) # YOOOOO
    #supports = supports.to(device) # YOOOOO

    #print("supports is:::::")
    #print(supports)
    # Step 2: gather embeddings from all workers
    supports = AllGather.apply(supports)

    # Step 3: compute similarlity between local embeddings
    #return softmax(query @ supports.T / tau) @ labels
    #return softmax(query @ supports.T / tau) @ labels
    ans = softmax(torch.matmul(query, supports.T / tau))

    print("ans shape is {}".format(ans.shape))
    #print(ans)
    print("labels shape is {}".format(labels.shape))
    #print(labels)

    #result = softmax(query @ supports.T / tau) @ labels
    result = torch.matmul(softmax(torch.matmul(query, supports.T / tau)), labels)
    #result = ans
    #print("result is:")

    return result

def my_loss_func(
    anchor_views,
    anchor_supports,
    anchor_support_labels,
    target_views,
    target_supports,
    target_support_labels,
    sharpen=sharpen_func,
    snn=snn
):
    # -- NOTE: num views of each unlabeled instance = 2+multicrop
    batch_size = len(anchor_views) // (2+multicrop)

    # Step 1: compute anchor predictions
    probs = snn(anchor_views, anchor_supports, anchor_support_labels, tau)

    # Step 2: compute targets for anchor predictions
    with torch.no_grad():
        targets = snn(target_views, target_supports, target_support_labels, tau)
        targets = sharpen(targets)
        if multicrop > 0:
            mc_target = 0.5*(targets[:batch_size]+targets[batch_size:])
            targets = torch.cat([targets, *[mc_target for _ in range(multicrop)]], dim=0)
        targets[targets < 1e-4] *= 0  # numerical stability

    # Step 3: compute cross-entropy loss H(targets, queries)
    #loss = torch.mean(torch.sum(torch.log(probs**(-targets)), dim=1))

    print("probs shape is {}".format(probs.shape))
    print("targets shape is {}".format(targets.shape))

    criterion = torch.nn.CrossEntropyLoss()
    targets2 = targets.argmax(-1)
    loss = criterion(probs, targets2)

    # Step 4: compute me-max regularizer
    rloss = 0.
    if me_max:
        avg_probs = AllReduce.apply(torch.mean(sharpen(probs), dim=0))
        rloss -= torch.sum(torch.log(avg_probs**(-avg_probs)))

    #return loss, rloss
    return loss, rloss

----

In [ ]:
##########################################################
##########################################################
################# DEBUG DATA LOADERS #####################
##########################################################
##########################################################

In [ ]:
##########################################################
##########################################################
##########################################################
##########################################################

----

In [49]:
SERIAL_EXEC = xmp.MpSerialExecutor()
# Only instantiate model weights once in memory.
#WRAPPED_MODEL = xmp.MpModelWrapper(ResNet18())

def train_resnet18():
  torch.manual_seed(1)

  ############# PAWS CODE ##################

  device = xm.xla_device()

  # -- init model
  encoder = init_model(
    device=device,
    model_name=model_name,
    use_pred=use_pred_head,
    output_dim=output_dim,
  )

  # -- init losses
  #paws = init_paws_loss(multicrop=multicrop, tau=temperature, T=sharpen, me_max=reg)
  # -- assume support images are sampled with ClassStratifiedSampler
  """
  labels_matrix = make_labels_matrix(
    num_classes=classes_per_batch,
    s_batch_size=s_batch_size,
    world_size=xm.xrt_world_size(),
    device=device,
    unique_classes=unique_classes,
    smoothing=label_smoothing,
  )
  """
  # TO DO: Hacer el smooth de los labels Matrix

  print("normalize {}".format(normalize))

# -- make data transforms
  transform, init_transform = make_transforms(
    dataset_name=dataset_name,
    subset_path=subset_path,
    unlabeled_frac=unlabeled_frac,
    training=True,
    split_seed=data_seed,
    crop_scale=crop_scale,
    basic_augmentations=False,
    color_jitter=color_jitter,
    normalize=normalize,
  )
  multicrop_transform = (multicrop, None)
  if multicrop > 0:
    multicrop_transform = make_multicrop_transform(
        dataset_name=dataset_name,
        num_crops=multicrop,
        size=mc_size,
        crop_scale=mc_scale,
        normalize=normalize,
        color_distortion=color_jitter,
    )

# -- init data-loaders/samplers
  (
    unsupervised_loader,
    unsupervised_sampler,
    supervised_loader,
    supervised_sampler,
) = init_data(
    dataset_name=dataset_name,
    transform=transform,
    init_transform=init_transform,
    supervised_views=supervised_views,
    u_batch_size=u_batch_size,
    s_batch_size=s_batch_size,
    unique_classes=unique_classes,
    classes_per_batch=classes_per_batch,
    multicrop_transform=multicrop_transform,
    world_size=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    root_path=root_path,
    s_image_folder=s_image_folder,
    u_image_folder=u_image_folder,
    training=True,
    copy_data=copy_data,
  )
  #iter_supervised = None
  ipe = len(unsupervised_loader)

  print("len unsuupervised dataloader IS: {}".format(ipe))


  logger.info(f"iterations per epoch: {ipe}")

    # -- init optimizer and scheduler
  scaler = torch.cuda.amp.GradScaler(enabled=use_fp16)

  """
  encoder, optimizer, scheduler = init_opt(
        encoder=encoder,
        weight_decay=wd,
        start_lr=start_lr,
        ref_lr=lr,
        final_lr=final_lr,
        ref_mom=mom,
        nesterov=nesterov,
        iterations_per_epoch=ipe,
        warmup=warmup,
        num_epochs=num_epochs,
    )
  """
  
  # YOOOO
  optimizer = torch.optim.Adam(encoder.parameters(), 0.0001, weight_decay=5e-4)
  # YOOOO
  
    #if xm.xrt_world_size() > 1:
    #    encoder = DistributedDataParallel(encoder, broadcast_buffers=False)

  def train_loop_fn(supervised_loader, unsupervised_loader, epoch):
      print("Entre a `train_loop_fn`")

      # -- TRAINING LOOP
      best_loss = None

      logger.info("Epoch %d" % (epoch + 1))

      """
      TO DO: Hacer un Double CHECK EN este Sampler!!!
      """
      # -- update distributed-data-loader epoch
      unsupervised_sampler.set_epoch(epoch)
      #if supervised_sampler is not None:
      #    supervised_sampler.set_epoch(epoch)

      loss_meter = AverageMeter()
      ploss_meter = AverageMeter()
      rloss_meter = AverageMeter()
      time_meter = AverageMeter()
      data_meter = AverageMeter()

      print("len supervised loader dentro de 'train_loop_fn` is {}".format(len(supervised_loader)))
      print("len unsupervised_loader loader dentro de 'train_loop_fn` is {}".format(len(unsupervised_loader)))

      for itr, udata in enumerate(unsupervised_loader):
          print("Entre a `for itr, udata`")
          def load_imgs(supervised_loader):
              # -- unsupervised imgs
              uimgs = [u.to(device, non_blocking=True) for u in udata[:-1]]
              # -- supervised imgs
              global iter_supervised
              print("len supervised loader is {}".format(len(supervised_loader)))
              try:
                  print("Entro al Try")
                  sdata = next(iter_supervised)
              except Exception:
                  print("Entro al Except!!")
                  iter_supervised = iter(supervised_loader)
                  logger.info(f"len.supervised_loader: {len(iter_supervised)}")
                  sdata = next(iter_supervised)
              finally:
                  idx = sdata[1].clone().detach()
                  idx = idx.to(device)

                  labels_matrix = torch.zeros(len(idx), idx.max()+1, device = device).scatter_(1, idx.unsqueeze(1), 1.)
  
                  labels_matrix = labels_matrix.to(device)
                  labels = torch.cat([labels_matrix for _ in range(supervised_views)])
                  simgs = [s.to(device, non_blocking=True) for s in sdata[:-1]]
              # -- concatenate supervised imgs and unsupervised imgs
              imgs = simgs + uimgs
              return imgs, labels
          
          imgs, labels = load_imgs(supervised_loader)
          
          def train_step():
              print("Entre a train_step()")

              #with torch.cuda.amp.autocast(enabled=use_fp16):
              optimizer.zero_grad()

              # --
              # h: representations of 'imgs' before head
              # z: representations of 'imgs' after head
              # -- If use_pred_head=False, then encoder.pred (prediction
              #    head) is None, and _forward_head just returns the
              #    identity, z=h
              h, z = encoder(imgs, return_before_head=True)

              # Compute paws loss in full precision
              #with torch.cuda.amp.autocast(enabled=False):

              # Step 1. convert representations to fp32
              h, z = h.float(), z.float()

              # Step 2. determine anchor views/supports and their
              #         corresponding target views/supports
              # --
              num_support = (
                  supervised_views * s_batch_size * classes_per_batch
              )

              print("num_support is {}".format(num_support))

              # --
              anchor_supports = z[:num_support]
              anchor_views = z[num_support:]
              # --
              target_supports = h[:num_support].detach()
              target_views = h[num_support:].detach()
              target_views = torch.cat(
                  [
                      target_views[u_batch_size : 2 * u_batch_size],
                      target_views[:u_batch_size],
                  ],
                  dim=0,
              )

              # Step 3. compute paws loss with me-max regularization
              print("# Step 3. compute paws loss with me-max regularization")

              #ploss, me_max ,lr_stats= 0,0,0 # FAKE LOSS 

              print("`anchor_views` shape is {}".format(anchor_views.shape))
              print("`anchor_supports` shape is {}".format(anchor_supports.shape))
              print("`labels shape` is {}".format(labels.shape))
              print("`target_views` shape is {}".format(target_views.shape))
              print("`target_supports` shape is {}".format(target_supports.shape))

              ploss, me_max = my_loss_func(
                  anchor_views=anchor_views,
                  anchor_supports=anchor_supports,
                  anchor_support_labels=labels,
                  target_views=target_views,
                  target_supports=target_supports,
                  target_support_labels=labels,
              )

              print("Termine Calculo de PAWS loss")

              loss = ploss + me_max

              print("loss is {}".format(loss))

              print("Doing loss backward")
              loss.backward()
              print("Optimizing step optimizer")
              xm.optimizer_step(optimizer)
              print("Finish Optimizer Step")
 
              return (float(loss), float(ploss), float(me_max))

          (loss, ploss, rloss) = train_step()
          #loss_meter.update(loss)
          #ploss_meter.update(ploss)
          #rloss_meter.update(rloss)

  ##########################################

  data, pred, target = None, None, None

  start_epoch = 0
  end_epoch = FLAGS.get("num_epochs")

  train_supervised_loader = pl.MpDeviceLoader(supervised_loader, device)
  train_unsupervised_loader = pl.MpDeviceLoader(unsupervised_loader, device)

  for epoch in range(start_epoch, end_epoch):
      train_loop_fn(train_supervised_loader, train_unsupervised_loader, epoch)
      xm.master_print("Finished training epoch {}".format(epoch))
  #return accuracy, data, pred, target
  return 0, 0, 0, 0

In [50]:
"""
To Do: 
  - Dont Donwload several Times ResNet model. How?. 
"""

'\nTo Do: \n  - Dont Donwload several Times ResNet model. How?. \n'

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
  global FLAGS
  FLAGS = flags
  torch.set_default_tensor_type('torch.FloatTensor')
  accuracy, data, pred, target = train_resnet18()

xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=FLAGS['num_cores'], start_method='fork')

normalize True
self.multicrop_transform (6, Compose(
    Compose(
    RandomResizedCrop(size=(96, 96), scale=(0.05, 0.14), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    Compose(
    RandomApply(
    p=0.8
    ColorJitter(brightness=[0.19999999999999996, 1.8], contrast=[0.19999999999999996, 1.8], saturation=[0.19999999999999996, 1.8], hue=[-0.2, 0.2])
)
    RandomGrayscale(p=0.2)
)
    ToTensor()
)
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
))
self.multicrop_transform (0, None)
len unsuupervised dataloader IS: 7
Entre a `train_loop_fn`
len supervised loader dentro de 'train_loop_fn` is 1
len unsupervised_loader loader dentro de 'train_loop_fn` is 7
normalize True
self.multicrop_transform (6, Compose(
    Compose(
    RandomResizedCrop(size=(96, 96), scale=(0.05, 0.14), ratio=(0.75, 1.3333), interpolation=bilinear)
    RandomHorizontalFlip(p=0.5)
    Compose(
    RandomApply(
    p=0.8
    ColorJitter(brightness=[0.19999999999